<a href="https://colab.research.google.com/github/Shravya12hi/AICTE_internship_2025/blob/main/Article_Generation_using_Multi_Agents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#---------------------------------
# Install required packages
#---------------------------------
# transformers  : for Hugging Face model loading and text generation
# fpdf          : for creating and saving PDF files
# markdown2     : for converting text to markdown format
!pip install transformers fpdf markdown2


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=a4c452e51d13344d54b86f85fa926a6808df17fe5ea692251412f17a412fb327
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [ ]:
# Python standard imports
# Provides functions for interacting with the operating system (file handling, paths, etc.
import os
# Used to handle JSON data (reading, writing, parsing)
import json
# Provides type hints such as Optional values and List data structures
from typing import Optional, List
# Used for data validation and structured models (here for the custom LLM wrapper)
from pydantic import BaseModel
# Provides date and time functionality
from datetime import datetime

# Capture and print the current timestamp (useful for logging or tracking runs)
timestamp = datetime.now()
print(timestamp)


2025-10-04 10:34:59.657360


In [ ]:
# LangChain imports

# Used to store and manage conversation history for each agent
from langchain.memory import ConversationBufferMemory

# Core building block that connects a language model with a prompt
from langchain.chains import LLMChain

# Helps define structured prompt templates for each agent
from langchain.prompts import PromptTemplate

# Base class for integrating custom language models into LangChain
from langchain.llms.base import LLM


In [ ]:
# Hugging Face imports

# AutoModelForSeq2SeqLM : Loads a pre-trained sequence-to-sequence model (used for text generation)
# AutoTokenizer         : Loads the tokenizer that converts text into tokens for the model
# pipeline              : Provides a simplified interface to run text generation tasks
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# PDF & Markdown

# FPDF      : Used to generate and save PDF documents
# markdown2 : Converts text content into Markdown format
from fpdf import FPDF
import markdown2

In [ ]:

# ------------------------
# Load Hugging Face Model
# ------------------------

# The pre-trained model being used is "google/flan-t5-large"
# This model is designed for text-to-text generation tasks
model_name = "google/flan-t5-large"

# Print a status message to indicate model loading
print("Loading Hugging Face model...")

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the sequence-to-sequence model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a text generation pipeline
# - task: text2text-generation (input text -> output text)
# - max_length: maximum length of generated text
# - max_new_tokens: maximum tokens generated beyond input
# - do_sample: enables sampling instead of greedy decoding
# - top_p: nucleus sampling, controls randomness
# - temperature: controls creativity in output (lower = more focused)
# - repetition_penalty: penalizes repeated phrases

text_generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2500,
    max_new_tokens=1200,
    do_sample=True,
    top_p=0.9,
    temperature=0.4,
    repetition_penalty=2.5
)


Loading Hugging Face model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Custom LLM wrapper for LangChain

# This class allows the Hugging Face pipeline to be used as a LangChain LLM
# Inherits from:
# - LLM        : Base LangChain class for language models
# - BaseModel  : Provides structured data validation from Pydantic
class HuggingFaceLLM(LLM, BaseModel):

# The Hugging Face text generation pipeline
    pipeline: object

# Identify the type of LLM for LangChain
    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

# Define how the model generates text given a prompt
# Optional stop tokens can be provided to control generation
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        outputs = self.pipeline(prompt)
        return outputs[0]['generated_text']

# Parameters that help identify this LLM instance
    @property
    def _identifying_params(self):
        return {"pipeline": str(self.pipeline)}

# Instantiate the custom LLM using the text generation pipeline
llm = HuggingFaceLLM(pipeline=text_generator)

In [ ]:
# Scratchpad to pass data between agents
# This acts as a simple key-value store accessible by all agents
shared_scratchpad = {}

# Function to update a value in the scratchpad
def update_scratchpad(key, value):
    shared_scratchpad[key] = value

# Function to retrieve a value from the scratchpad
# Returns an empty string if the key does not exist
def get_scratchpad(key):
    return shared_scratchpad.get(key, "")

# Memories for each agent
# ConversationBufferMemory stores the interaction history for each agent
# This allows agents to remember past interactions and context
research_memory = ConversationBufferMemory(memory_key="history", return_messages=True)
grammar_memory = ConversationBufferMemory(memory_key="history", return_messages=True)
seo_memory = ConversationBufferMemory(memory_key="history", return_messages=True)
humanize_memory = ConversationBufferMemory(memory_key="history", return_messages=True)
publish_memory = ConversationBufferMemory(memory_key="history", return_messages=True)


/tmp/ipython-input-1339270976.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  research_memory = ConversationBufferMemory(memory_key="history", return_messages=True)


In [ ]:
# List to store logs of all agent action
logs = []

# Function to log actions performed by each agent
# Records the agent name, a summary of its action, and a timestamp
def log_agent_action(agent_name, summary=""):
    entry = {
        "timestamp": datetime.now().isoformat(),  # Current timestamp in ISO format
        "agent": agent_name,                      # Name of the agent performing the action
        "summary": summary                        # Short description of what the agent did
    }
    # Append the log entry to the logs list
    logs.append(entry)

    # Optional: save logs to file each time
    # This allows tracking the pipeline progress even if the session ends
    with open("pipeline_logs.json", "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)


In [ ]:
# Define the prompt template for the Research Agent
# The Research Agent generates the initial draft of the article based on the topic

research_prompt = PromptTemplate(
    input_variables=["topic"],       # The variable that will be inserted into the template
    template=(
      "Write a comprehensive article on the topic: \"{topic}\".\n\n"
        "Requirements:\n"
        "- Start with a natural, engaging introduction (do not say 'this is an introduction').\n"
        "- Provide 3–5 detailed body paragraphs with examples, evidence, and reasoning.\n"
        "- Conclude with a strong closing paragraph.\n"
        "- Use clear transitions between ideas.\n"
        "- Write at least 800 words in full sentences.\n"
        "Do not describe the structure; just write the article directly."

    )
)


# Create the LLMChain for the Research Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in memory/scratchpad
# - memory: keeps a history of previous interactions for context
research_chain = LLMChain(
    llm=llm,
    prompt=research_prompt,
    output_key="research_output",
    memory=research_memory
)



/tmp/ipython-input-3693368830.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  research_chain = LLMChain(


In [ ]:
# Define the prompt template for the Grammar Agent
# The Grammar Agent improves the readability and correctness of the article

grammar_prompt = PromptTemplate(
    input_variables=["article"],    # The variable that will be inserted into the template
    template=(
         "You are a Grammar and Readability Editor.\n\n"
        "Improve the article by:\n"
        "- Correcting grammar and spelling.\n"
        "- Improving sentence clarity and flow.\n"
        "- Enhancing readability with smooth transitions.\n"
        "- Keeping the meaning intact.\n\n"
        "Return only the improved article, not comments or explanations.\n\n"
        "Article:\n{article}"
    )
)

# Create the LLMChain for the Grammar Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in memory/scratchpad
# - memory: keeps a history of previous interactions for context
grammar_chain = LLMChain(
    llm=llm,
    prompt=grammar_prompt,
    output_key="grammar_output",
    memory=grammar_memory
)

In [ ]:
# Define the prompt template for the SEO Agent
# The SEO Agent optimizes the article for search engines while keeping it readable

seo_prompt = PromptTemplate(
    input_variables=["article", "keywords"],  # Variables to be inserted into the template
    template=(
        "You are an SEO Optimizer.\n\n"
        "Optimize the article while keeping it natural and readable:\n"
        "- Insert the keywords naturally, avoid keyword stuffing.\n"
        "- Maintain ~1–2% keyword density.\n"
        "- Generate an SEO-friendly title (<60 characters).\n"
        "- Add a short meta description (150–160 characters).\n"
        "- Use H1, H2, and H3 headings with keywords.\n"
        "- Do not explain changes, only return the optimized article with headings included.\n\n"
        "Article:\n{article}\n"
        "Keywords:\n{keywords}"
    )
)

# Create the LLMChain for the SEO Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in scratchpad (here None since memory is not used)
# - memory: not used for SEO agent
seo_chain = LLMChain(
    llm=llm,
    prompt=seo_prompt,
    output_key="seo_output",
    memory=None
)

In [ ]:
# Define the prompt template for the Humanizer Agent
# The Humanizer Agent rewrites the article to make it more engaging, relatable, and easier to read

humanize_prompt = PromptTemplate(
    input_variables=["article"],  # Variable to be inserted into the template
    template=(
        "Rewrite the article to make it more engaging, relatable, and easy to read.\n"
        "- Use natural language and storytelling elements.\n"
        "- Add examples where helpful.\n"
        "- Keep it professional but approachable.\n"
        "- Do not explain what you are doing, just return the rewritten article.\n\n"
        "Article:\n{article}"
    )
)

# Create the LLMChain for the Humanizer Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in memory/scratchpad
# - memory: keeps a history of previous interactions for context
humanize_chain = LLMChain(
    llm=llm,
    prompt=humanize_prompt,
    output_key="humanize_output",
    memory=humanize_memory
)

In [ ]:
# Define the prompt template for the Publisher Agent
# The Publisher Agent finalizes the article and formats it for publishing
publish_prompt = PromptTemplate(
    input_variables=["article"],  # Variable to be inserted into the template
    template=(
        "Finalize the article for publishing.\n\n"
        "Return the result in **JSON** format with two keys:\n"
        "1. article → the clean final article text.\n"
        "2. metadata → object with:\n"
        "   - filename (string)\n"
        "   - timestamp (string)\n"
        "   - tags (list of 3–5 relevant keywords)\n"
        "   - summary (one sentence under 50 words)\n\n"
        "Do not add explanations. Output JSON only.\n\n"
        "Article:\n{article}"
    )
)


# Create the LLMChain for the Publisher Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in memory/scratchpad
# - memory: keeps a history of previous interactions for context
publish_chain = LLMChain(
    llm=llm,
    prompt=publish_prompt,
    output_key="publish_output",
    memory=publish_memory
)

In [ ]:
# Define the prompt template for the Feedback Agent
# The Feedback Agent rewrites the article based on given feedback while keeping the meaning intact

feedback_prompt = PromptTemplate(
    input_variables=["article", "feedback"], # Variables to be inserted into the template
    template=(
        "You are an expert editor. Rewrite the following article completely based on the feedback "
        "while keeping the meaning intact. Ensure the final article has:\n"
        "- A clear introduction\n"
        "- 3 to 5 well-structured body paragraphs\n"
        "- A strong conclusion\n"
        "- Full, grammatically correct sentences (no fragments)\n"
        "- Minimum 400 words\n\n"
        "Feedback: {feedback}\n\n"
        "Original Article:\n{article}\n\n"
        "Return the full rewritten article only."
    )
)


# Create the LLMChain for the Feedback Agent
# - llm: the HuggingFaceLLM instance
# - prompt: the prompt template defined above
# - output_key: key used to store the output in memory/scratchpad
# - memory: not used here, but could track feedback history
feedback_chain = LLMChain(
    llm=llm,
    prompt=feedback_prompt,
    output_key="refined_article",
    memory=None  # Optional, can track feedback history
)

In [ ]:
# --------------------------
# Helper functions
# --------------------------
def apply_feedback_list(article: str, feedback_list: list):
  # Start with the original article
    refined_article = article

  # Loop through each feedback item and apply it using the Feedback Agent
    for feedback in feedback_list:
      # Generate a refined article based on current feedback
        refined_article = feedback_chain.run(article=refined_article, feedback=feedback).strip()

      # Update the scratchpad with the latest humanized version
        update_scratchpad("humanize", refined_article)

  # Return the fully refined article after all feedback has been applied
    return refined_article


In [ ]:
# Helper function to save the final article as both Markdown and PDF
def save_article(article: str, metadata: dict):
    # Determine the base filename from metadata or generate one using the current timestamp
    filename_base = metadata.get("filename", f"article_{datetime.now().strftime('%Y%m%d_%H%M%S')}")


    # Save Markdown
    md_file = f"{filename_base}.md"
    with open(md_file, "w", encoding="utf-8") as f:
        f.write(article)

    # Save PDF
    pdf_file = f"{filename_base}.pdf"
    pdf = FPDF()                   # Initialize PDF object
    pdf.add_page()                 # Add a new page
    pdf.set_font("Arial", size=12) # Set font type and size

    # Write each line of the article into the PDF
    for line in article.split("\n"):
        pdf.multi_cell(0, 10, line)

    # Export the PDF file
    pdf.output(pdf_file)

    # Return a dictionary with the filenames of saved Markdown and PDF files
    return {"markdown_file": md_file, "pdf_file": pdf_file}


In [ ]:
# Function to run the Research Agent
# Generates the initial draft of the article based on the provided topic
def run_research(topic):
    # Run the Research Agent chain with the given topic
    result = research_chain.run(topic=topic)

    # Store the generated article in the scratchpad for use by other agents
    update_scratchpad("research", result)

    # Return the generated article
    return result

In [ ]:
# Install and import textstat library for readability analysis
!pip install textstat
import textstat

# Function to run the Grammar Agent
# Improves grammar, clarity, and readability of the article
def run_grammar():
  # Retrieve the article generated by the Research Agent from the scratchpad
    article = get_scratchpad("research")
  # Run the Grammar Agent chain to improve the article
    result = grammar_chain.run(article=article)
  # Store the improved article in the scratchpad for further processing
    update_scratchpad("grammar", result)

  # Calculate readability score using Flesch Reading Ease
    readability_score = textstat.flesch_reading_ease(result)
  # Log the action and the readability score
    log_agent_action("Grammar Agent", f"Readability score: {readability_score}")

  # Return the improved article
    return result


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.0 MB/s eta 0:00:00


In [ ]:
# Function to calculate the keyword density in an article
# Returns a dictionary mapping each keyword to its percentage occurrence
def calculate_keyword_density(article, keywords):
    # Split article into lowercase words
    words = article.lower().split()
    density = {}

    # Calculate density for each keyword
    for kw in keywords.split(","):
        kw = kw.strip().lower()       # Clean whitespace and convert to lowercase
        count = words.count(kw)       # Count occurrences of the keyword
        density[kw] = round(count / len(words) * 100, 2) # Calculate percentage
    return density


# Function to run the SEO Agent
# Optimizes the article for SEO using provided keywords
def run_seo(keywords):
  # Retrieve the article processed by the Grammar Agent
    article = get_scratchpad("grammar")

  # Run the SEO Agent chain with the article and keywords
    result = seo_chain.run({"article": article, "keywords": keywords})

  # Store the SEO-optimized article in the scratchpad
    update_scratchpad("seo", result)

  # Calculate keyword density in the optimized article
    density = calculate_keyword_density(result, keywords)

  # Log the SEO action and keyword density
    log_agent_action("SEO Agent", f"Keyword density: {density}")
  # Return the SEO-optimized article
    return result

In [ ]:
# Function to run the Humanizer Agent
# Rewrites the article to make it more engaging, relatable, and readable
def run_humanize():
  # Retrieve the article processed by the SEO Agent
    article = get_scratchpad("seo")

  # Run the Humanizer Agent chain on the article
    result = humanize_chain.run(article=article)

  # Store the humanized article in the scratchpad
    update_scratchpad("humanize", result)

  # Return the humanized article
    return result

In [ ]:
def run_publish(article: str):
    """
    Finalizes the article: applies metadata, generates summary if missing,
    and saves as Markdown and PDF. Fully automated, no dialog.
    """
    final_article = article
    # Try to parse metadata from the Publisher Agent output
    try:
        result = publish_chain.run(article=article)
        parsed = json.loads(result)
        metadata = parsed.get("metadata", {})           # Extract metadata
        final_article = parsed.get("article", article)  # Use article from JSON if available
    except:
        # Fallback in case JSON parsing fails
        metadata = {}

    # --------------------------
    # Ensure metadata fields exist
    # --------------------------
    metadata.setdefault("filename", f"article_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    metadata.setdefault("timestamp", str(datetime.now()))
    metadata.setdefault("tags", ["general"])

    # Generate a short summary (first 500 words) if missing
    if not metadata.get("summary"):
        words = final_article.split()
        summary = " ".join(words[:500]) if words else "No content available."
        metadata["summary"] = summary

    # --------------------------
    # Save article as Markdown and PDF
    # --------------------------
    saved_files = save_article(final_article, metadata)

    # Print saved file information and summary
    print(f"✅ Article saved as Markdown and PDF:\n- {saved_files['markdown_file']}\n- {saved_files['pdf_file']}")
    print(f"📄 Summary: {metadata['summary']}")

    # Return the finalized article and metadata
    return final_article, metadata


In [ ]:
# Main execution block
if __name__ == "__main__":
    # Define the topic and SEO keywords
    topic_input = "The impact of artificial intelligence on education"
    keywords_input = "AI, Education, Artificial Intelligence, Learning, Schools"  # Manual SEO keywords

    # Store topic in scratchpad for agent access
    update_scratchpad("topic", topic_input)

#1. Research Agent
# Generate the initial draft of the article
    print("1️⃣ Research Agent generating article...")
    research_result = run_research(topic_input)
    log_agent_action("Research Agent", research_result[:100] + "...")

#2. Grammar Agent
# Improve grammar, clarity, and readability
    print("2️⃣ Grammar Agent improving article...")
    grammar_result = run_grammar()
    log_agent_action("Grammar Agent", f"Improved article; Readability score calculated.") # Log first 100 characters

#3. SEO optimizer
# Enhance the article for SEO with keywords
    print("3️⃣ SEO Optimizer enhancing article...")
    seo_result = run_seo(keywords_input)  # This already calculates keyword density
    log_agent_action("SEO Agent", "SEO optimization applied with keyword density logged.")

#4. Humanizer
# Make the article more engaging and relatable
    print("4️⃣ Humanizer making article relatable...")
    humanized_result = run_humanize()
    log_agent_action("Humanizer Agent", "Article humanized and made more engaging.")


#5. Applying predefined feedback refinements
# Refine the article with specific feedback
    print("5️⃣ Applying predefined feedback refinements automatically...")
    feedback_list = [
        "Add a concrete example of AI helping students in a classroom.",
        "Add a concrete example of AI helping teachers plan lessons.",
        "Make the tone slightly more formal."
    ]
    refined_article = get_scratchpad("humanize")
    refined_article = apply_feedback_list(refined_article, feedback_list)

# Log each feedback applied
    for fb in feedback_list:
        log_agent_action("Feedback Agent", fb)

#6. Publisher
# Finalize the article and save as Markdown and PDF
    print("6️⃣ Publisher finalizing article...")
    final_article, metadata = run_publish(refined_article)

# ✅ Pipeline Completed
# Print final metadata
    print("\n--- Pipeline Completed ---")
    print("Filename:", metadata["filename"])
    print("Summary:", metadata["summary"])


1️⃣ Research Agent generating article...


/tmp/ipython-input-1537867212.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = research_chain.run(topic=topic)


2️⃣ Grammar Agent improving article...
3️⃣ SEO Optimizer enhancing article...
4️⃣ Humanizer making article relatable...
5️⃣ Applying predefined feedback refinements automatically...
6️⃣ Publisher finalizing article...
✅ Article saved as Markdown and PDF:
- article_20251004_105112.md
- article_20251004_105112.pdf
📄 Summary: Artificial intelligence is becoming more commonplace in classrooms around the world. As technology becomes more advanced, it will have a greater impact on teaching and learning than ever before. For example, by 2025, one in five teachers will be expected to teach using computer-aided technology. By 2050, this number will increase to over 50%. One of the biggest challenges facing schools today is the lack of skilled teachers who can deal with the increasing use of AI. To address this problem, educators need to develop innovative ways to incorporate AI into their curriculum.

--- Pipeline Completed ---
Filename: article_20251004_105112
Summary: Artificial intelligence 